# Funcionalidades

In [1]:
from ultralytics import YOLO

In [19]:
# Cargar modelo YOLO11s fine-tuned
model = YOLO('C:\\Users\\alvar\\Desktop\\Proyecto G3\\ProyectoGrupal-G3-UEM\\modelos\\best.pt')

##  Cantidad de clientes por zona

In [14]:
# Funcion para determinar coordenadas de objetos en videos
import cv2

# Función de callback para obtener coordenadas al hacer clic
def obtener_coordenadas(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:  # Solo muestra al hacer clic izquierdo
        print(f'Coordenadas: ({x}, {y})')

# Capturar un frame del video
cap = cv2.VideoCapture('C:\\Users\\alvar\\Desktop\\Proyecto G3\\ProyectoGrupal-G3-UEM\\videos camaras seleccionadas\\Canal2.mp4')
ret, frame = cap.read()
cap.release()

if ret:
    cv2.imshow('Frame', frame)
    cv2.setMouseCallback('Frame', obtener_coordenadas)  # Asigna la función al frame

    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("No se pudo leer el frame del video.")



Coordenadas: (476, 251)
Coordenadas: (787, 562)


In [12]:
import numpy as np
from datetime import datetime, timedelta
import pandas as pd

In [4]:
# Función para calcular la distancia entre un punto (px, py) y una línea definida por dos puntos (x1, y1) y (x2, y2)
def distancia_punto_linea(px, py, x1, y1, x2, y2):
    return abs((y2 - y1) * px - (x2 - x1) * py + x2 * y1 - y2 * x1) / np.sqrt((y2 - y1)**2 + (x2 - x1)**2)

In [5]:
# Definir las coordenadas de la línea del mostrador (inicio y fin)
x1_mostrador, y1_mostrador = 476, 251
x2_mostrador, y2_mostrador = 787, 562

In [26]:
# Función para contar los clientes cerca de la línea del mostrador
def contar_clientes_linea(detecciones, x1_mostrador, y1_mostrador, x2_mostrador, y2_mostrador):
    count = 0
    for deteccion in detecciones:
        # Extraer las coordenadas (xywh) y la confianza
        bbox = deteccion.xywh[0]  # xywh de la detección
        x, y, w, h = bbox[0], bbox[1], bbox[2], bbox[3]  # Extraemos las coordenadas y el tamaño
        
        # Calcular el centro del objeto (usamos las coordenadas en formato xywh)
        centro_x, centro_y = x + w / 2, y - h / 2  # Centro del objeto

        # Calcular la distancia entre el centro del objeto y la línea del mostrador
        distancia = distancia_punto_linea(centro_x, centro_y, x1_mostrador, y1_mostrador, x2_mostrador, y2_mostrador)

        # Si la distancia es pequeña, consideramos que el cliente está en la zona del mostrador
        if distancia < 50:  # Ajusta el umbral según sea necesario
            count += 1

    return count


In [17]:
#  funcion para ver la Linea que define al mostrador: 
video = cv2.VideoCapture("C:\\Users\\alvar\\Desktop\\Proyecto G3\\ProyectoGrupal-G3-UEM\\videos camaras seleccionadas\\Canal2.mp4")

while True:
    ret, frame = video.read()
    if not ret:
        break

    # Pintar la línea en el frame
    color = (0, 255, 0)  # Color de la línea (verde en este caso)
    grosor = 2  # Grosor de la línea
    cv2.line(frame, (x1_mostrador, y1_mostrador), (x2_mostrador, y2_mostrador), color, grosor)

    # Mostrar el frame con la línea
    cv2.imshow('Frame con Linea Mostrador', frame)

    # Salir si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar el video y cerrar la ventana
video.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [10]:
# Leer video
import cv2
video = cv2.VideoCapture("C:\\Users\\alvar\\Desktop\\Proyecto G3\\ProyectoGrupal-G3-UEM\\grabaciones\\grabacion_2025-02-21_19-42-20.mp4")

# Lista para almacenar los resultados
resultados = []

# Extraer la fecha y hora inicial desde el nombre del archivo
nombre_video = "grabacion_2025-02-21_19-42-20.mp4"
fecha_hora_inicial_str = nombre_video.split('_')[1] + " " + nombre_video.split('_')[2].replace('.mp4', '')
fecha_hora_inicial = datetime.strptime(fecha_hora_inicial_str, "%Y-%m-%d %H-%M-%S")

In [27]:
while True:
    ret, frame = video.read()
    if not ret:
        break

    # Obtener el número de fotograma y calcular el tiempo en segundos
    frame_number = int(video.get(cv2.CAP_PROP_POS_FRAMES))
    fps = video.get(cv2.CAP_PROP_FPS)
    segundos_transcurridos = frame_number / fps

    # Calcular la nueva fecha y hora basándonos en la fecha de inicio
    nueva_fecha_hora = fecha_hora_inicial + timedelta(seconds=segundos_transcurridos)

    # Formatear la nueva fecha y hora
    fecha_hora_exacta = nueva_fecha_hora.strftime("%Y-%m-%d %H:%M:%S")

    # Realizar la detección de objetos (clientes y empleados) usando el modelo YOLOv11s
    resultados_deteccion = model(frame)  # Detecta los objetos en el fotograma

    # Extraer las detecciones de los objetos
    detecciones = resultados_deteccion[0].boxes  # Obtenemos las cajas de las detecciones

    # Contar clientes cerca de la línea del mostrador
    clientes_mostrador = contar_clientes_linea(detecciones, x1_mostrador, y1_mostrador, x2_mostrador, y2_mostrador)

    # Guardar los resultados en la lista
    resultados.append([fecha_hora_exacta, segundos_transcurridos, clientes_mostrador])

# Guardar los resultados en un archivo CSV
df = pd.DataFrame(resultados, columns=['Fecha Hora', 'Segundo', 'Clientes Zona Mostrador'])
df.to_csv('resultados_deteccion_mostrador.csv', index=False)

# Liberar el video y cerrar la ventana
video.release()
cv2.destroyAllWindows()

0: 384x640 1 caja, 3 clientes, 1 mostrador, 447.5ms
Speed: 7.6ms preprocess, 447.5ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 3 clientes, 1 mostrador, 388.0ms
Speed: 5.1ms preprocess, 388.0ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 3 clientes, 443.3ms
Speed: 2.7ms preprocess, 443.3ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 3 clientes, 1 mostrador, 413.7ms
Speed: 3.2ms preprocess, 413.7ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 3 clientes, 1 mostrador, 367.5ms
Speed: 2.3ms preprocess, 367.5ms inference, 4.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 3 clientes, 1 mostrador, 318.9ms
Speed: 2.9ms preprocess, 318.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 caja, 4 clientes, 1 mostrador, 288.1ms
Speed: 2.1ms preprocess, 288.1ms inference, 1.5ms postp